In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql import Window as wn

In [2]:
spark = SparkSession.builder.master("local[2]").appName("Min MAx Salary").getOrCreate()

In [5]:
emp_data = [
    ('Siva',1,30000),
    ('Ravi',2,40000),
    ('Prasad',1,50000),
    ('Arun',1,30000),
    ('Sai',2,20000)
]
emp_schema = "emp_name string , dep_id int , salary long"
df = spark.createDataFrame(data = emp_data , schema = emp_schema)
df.show()

+--------+------+------+
|emp_name|dep_id|salary|
+--------+------+------+
|    Siva|     1| 30000|
|    Ravi|     2| 40000|
|  Prasad|     1| 50000|
|    Arun|     1| 30000|
|     Sai|     2| 20000|
+--------+------+------+



In [16]:
part_df = df.withColumn("DRN",FS.dense_rank().over(wn.partitionBy("dep_id").orderBy("salary","emp_name")))
part_df.show()

+--------+------+------+---+
|emp_name|dep_id|salary|DRN|
+--------+------+------+---+
|    Arun|     1| 30000|  1|
|    Siva|     1| 30000|  2|
|  Prasad|     1| 50000|  3|
|     Sai|     2| 20000|  1|
|    Ravi|     2| 40000|  2|
+--------+------+------+---+



In [20]:
part_df.groupBy("dep_id").agg(
    FS.when(FS.max(FS.col('DRN')) == 1,FS.col('emp_name')).alias('min'),
     FS.when(FS.min(FS.col('DRN')) == FS.col('dep_id'),FS.col('emp_name')).alias('max')
                             
                             ).show()

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "emp_name" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(emp_name)" if you do not care which of the values within a group is returned.;
Aggregate [dep_id#13], [dep_id#13, CASE WHEN (max(DRN#205) = dep_id#13) THEN emp_name#12 END AS min#266, CASE WHEN (min(DRN#205) = dep_id#13) THEN emp_name#12 END AS max#268]
+- Project [emp_name#12, dep_id#13, salary#14L, DRN#205]
   +- Project [emp_name#12, dep_id#13, salary#14L, DRN#205, DRN#205]
      +- Window [dense_rank(salary#14L, emp_name#12) windowspecdefinition(dep_id#13, salary#14L ASC NULLS FIRST, emp_name#12 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS DRN#205], [dep_id#13], [salary#14L ASC NULLS FIRST, emp_name#12 ASC NULLS FIRST]
         +- Project [emp_name#12, dep_id#13, salary#14L]
            +- Lo